Bazujemy na funkcjach rysujących stworzonych przy okazji poprzednich laboratoriów przez Tomasza Michalca:
https://github.com/Chimstaz/PyCXlab
Zadanie:
--
* Uzupełnienie następujących funkcji: 
 * zjadanie
 * reprodukcja
 * zwrócenie wszystkich możliwych sąsiadów
 * zachowanie poszczególnej komórki
* Zasady:
  * Trzy rodzaje komórek, ofiara drapieżnik i komórka pusta
  * Bazujemy na sąsiedztwie Moore'a
  * Osobniki poruszają się w losowe strony
  * Ofiara:
   * Zdrowie rośnie w miarę poruszania sie
   * Kiedy zdrowie osiągnie pewien próg tworzą nowego osobnika ( ofiarę lub drapieżnika z równym prawdopodobieństwem) i ich zdrowie resetuje się do bazowego
  * Drapieżnik:
   * Zdrowie maleje w miarę poruszania się 
   * Gdy spotkają ofiarę zjadają ją przejmując jej zdrowie

In [ ]:
import random

import pylab

import pycxsimulator

random.seed(None)


class Cell:

    def __init__(self, spec=None):
        self.species = spec if spec is not None else 0
        self.x = None
        self.y = None
        self.health = self.species * 2

    def set_location(self, pos_x, pos_y):
        self.x = pos_x
        self.y = pos_y

    def set_species(self, num):
        health_by_species = [0,2,4]
        self.species, self.health = (num, health_by_species[num]) if num in [0,1,2] else (0, 0)

    def move_here(self, species, health):
        self.species, self.health = (species, health) if species in [0,1,2] else (0, 0)

    def prey_eat(self):
        pass

    def prey_reproduce(self):
        pass
    

In [ ]:
world = []
height = 100
width = 100
empty_val = 0
predator_val = 1
prey_val = 2

def init():
    global time, world
    time = 0

    for x in range(width):
        row = []
        for y in range(height):
            i = random.uniform()
            if i <= 0.6:
                row.append(Cell(prey_val))
            elif i <= 0.8:
                row.append(Cell(predator_val))
            else:
                row.append(Cell(empty_val))
        world.append(row)

def step():
    global time, world
    time += 1
    for x in range(width):
        for y in range(height):
            check_neighbors(x,y,world[x][y])

def check_neighbors(x,y,cell):
    pass

def check_neighbour(x,y):
    pass

def draw():
    to_draw = []
    for x in range(width):
        row = []
        for y in range(height):
            row.append(world[x][y].species)
        to_draw.append(row)
    pylab.cla()
    pylab.pcolor(to_draw, vmin=0, vmax=2, cmap=pylab.cm.get_cmap("binary"))
    pylab.axis('image')
    pylab.title('t = ' + str(time))


pycxsimulator.GUI().start(func=[init, draw, step])


